In [1]:
# From:
#   https://hdfgroup.org/wp/2015/03/from-hdf5-datasets-to-apache-spark-rdds/
# Specialized to MIDS W205-2 (Fall 2015)

import os
import os.path
import h5py
import sys

mssRoot = os.path.expanduser('~/MIDS/mss/data/')
blocksRoot = os.path.expanduser('~/MIDS/mss/blocks/')
filesPerBlock = 100  # Gives about 30MB/file. Should use larger size in production.

minPartitions = 2 #minimum number of partitions for use by Spark


In [2]:
# Configure spark to work
import os
import sys

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))

#execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

In [3]:
def getFileNames(dataRoot, verbose = True):
	
	fileStructure = os.walk(dataRoot)
	
	h5Files = []
	for entry in fileStructure:
		thisPath = entry[0]
		thisFileList = entry[2]
		#print thisPath, thisFileList

		for ff in thisFileList:
			if ff.endswith('.h5'):
				h5Files.append(os.path.join(thisPath,ff))
				if verbose:
					print os.path.join(thisPath,ff)
	return h5Files

In [7]:
def buildReferenceCSV(dataRoot = None, publish = True, verbose = True):
	'''
	"Walks" the file tree and finds all of the HD5 files in the tree under dataRoot
	(assumes these are from the million song database)
	'''
	
	if not dataRoot:
		dataRoot = blocksRoot
    
	h5Files = getFileNames(dataRoot, verbose = verbose)

	if publish:
		csvFileName = os.path.join(dataRoot,'Hd5Extracts.csv')
		oFile = open(csvFileName, 'w')
        
	
	for ff in h5Files:

		pubString = ff
#		pubString += ', '
#		pubString += 'analysis, '
#		pubString += 'key \n'   # Key of the song, not key of the hash

		if publish:
			oFile.write(pubString + '\n')

		if verbose:
			print pubString

	return csvFileName

In [29]:
def nodeToList(inputLine, verbose = False):
    '''
    This function is intended for use in Spark.
    Inputs a line (probably from the reference CSV)
    and extracts some important info in an annoylingy
    hard-coded way [we should think about how to fix this]
    
    Returns contents of original input line with key,
    time_signature, and tempo appended.
    
    '''
    import h5py
    parsedLine = inputLine.split(',')
    thisH5File = h5py.File(parsedLine[0])
    thisBlockSongList = thisH5File['songs']
    
    songsWithData = []
    
    for thisSong in thisBlockSongList:
        dataSet = thisH5File['songs'][thisSong]['analysis']['songs']
        key = dataSet[0][-10]
        time_signature = dataSet[0][-3]
        tempo = dataSet[0][-4]
    
        outTuple = (parsedLine[0], key, time_signature, tempo) 
        
        #if verbose:
        print outTuple
        songsWithData.append(outTuple)
        
    return songsWithData


In [30]:
#  Process in Spark

from pyspark import SparkContext
from optparse import OptionParser
from datetime import datetime as dt
print 'Establishing Spark Context'

# Kill the context if it already exists
if 'sc' in dir():
    sc.stop()

sc = SparkContext(appName="SparkHDF5")

csvFileName = buildReferenceCSV(dataRoot = blocksRoot, verbose = False)

print 'Fetching all file names'
timeNow = dt.now().isoformat()
file_paths = sc.textFile(os.path.join(blocksRoot,'Hd5Extracts.csv'),minPartitions=minPartitions)

print 'Mapping RDD'
rdd = file_paths.flatMap(nodeToList)

sc.stop()

#print 'First Result: ', rdd.take(1)


Establishing Spark Context
Fetching all file names
Mapping RDD


In [31]:
rdd.take(1)

Py4JJavaError: An error occurred while calling o146.partitions.
: java.io.IOException: java.lang.NullPointerException
	at org.apache.spark.util.Utils$.tryOrIOException(Utils.scala:1178)
	at org.apache.spark.broadcast.TorrentBroadcast.readBroadcastBlock(TorrentBroadcast.scala:165)
	at org.apache.spark.broadcast.TorrentBroadcast._value$lzycompute(TorrentBroadcast.scala:64)
	at org.apache.spark.broadcast.TorrentBroadcast._value(TorrentBroadcast.scala:64)
	at org.apache.spark.broadcast.TorrentBroadcast.getValue(TorrentBroadcast.scala:88)
	at org.apache.spark.broadcast.Broadcast.value(Broadcast.scala:70)
	at org.apache.spark.rdd.HadoopRDD.getJobConf(HadoopRDD.scala:144)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:200)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
	at scala.Option.getOrElse(Option.scala:120)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:237)
	at org.apac

In [21]:
os.path.join(blocksRoot,'Hd5Extracts.csv')

'/home/james/MIDS/mss/blocks/Hd5Extracts.csv'

In [26]:
nodeToList('/home/james/MIDS/mss/blocks/Chunk0032.h5')

[('/home/james/MIDS/mss/blocks/Chunk0032.h5', 0, 1, 176.12200000000001),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 2, 7, 185.453),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 3, 4, 94.161000000000001),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 0, 3, 151.19499999999999),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 5, 4, 90.953999999999994),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 0, 4, 105.994),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 10, 4, 88.015000000000001),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 7, 1, 74.575999999999993),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 2, 3, 83.991),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 11, 5, 83.320999999999998),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 3, 4, 209.14099999999999),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 11, 4, 101.37),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 0, 4, 152.899),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 9, 4, 86.308000000000007),
 ('/home/james/MIDS/mss/blocks/C

In [24]:
inputLine = '/home/james/MIDS/mss/blocks/Chunk0032.h5'

In [25]:
nodeToList(inputLine)

[('/home/james/MIDS/mss/blocks/Chunk0032.h5', 0, 1, 176.12200000000001),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 2, 7, 185.453),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 3, 4, 94.161000000000001),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 0, 3, 151.19499999999999),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 5, 4, 90.953999999999994),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 0, 4, 105.994),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 10, 4, 88.015000000000001),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 7, 1, 74.575999999999993),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 2, 3, 83.991),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 11, 5, 83.320999999999998),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 3, 4, 209.14099999999999),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 11, 4, 101.37),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 0, 4, 152.899),
 ('/home/james/MIDS/mss/blocks/Chunk0032.h5', 9, 4, 86.308000000000007),
 ('/home/james/MIDS/mss/blocks/C